In [31]:
import networkx as nx
import csv
import numpy as np
import glob
import os, os.path
import heapq
import networkit as nk
import random
import collections
from random import sample
import re
import math
import time
import networkx.algorithms.community as nxcom
from matplotlib import pyplot as plt
from collections import OrderedDict
from networkx.generators.community import LFR_benchmark_graph

In [32]:
#function that approximates the diameter of the graph
def approxVD(G):
    node_number = G.number_of_nodes()
    random_vertice = random.choice(list(G.nodes))
    shortest_paths = nx.single_source_shortest_path(G, random_vertice)
    ordered_index = sorted(shortest_paths, key = lambda key: len(shortest_paths[key]))
    last_index = ordered_index[len(ordered_index) - 1]
    last_index2 = ordered_index[len(ordered_index) - 2]
    max_shortest1 = shortest_paths.get(last_index)
    max_shortest2 = shortest_paths.get(last_index2)
    diameter = len(max_shortest1) + len(max_shortest2)
    return diameter

In [33]:
def bcApproxGen(G, epsilon=0.3, delta=0.5, universalConstant=0.5):
    bc = dict()
    edges = list(G.edges)
    for edge in edges:
        bc[edge] = 0
    vd = (approxVD(G))
    r = (universalConstant/(epsilon*epsilon)) * (math.log((vd-2),2) + math.log(1/delta))
    for i in range(round(r+1)):
        while True:
            sample_pair = sample(list(G.nodes()), 2)
            if (nx.has_path(G,sample_pair[0],sample_pair[1])):
                break
        shortest_paths = nx.all_shortest_paths(G, source = sample_pair[0], target = sample_pair[1])
        shortest_paths = list(shortest_paths)
        sample_path = random.choice(shortest_paths)
        prev = sample_path[0]
        for j in range(1,len(sample_path)):
            try:
                bc[(sample_path[j],prev)] += 1/r
            except:
                bc[(prev,sample_path[j])] += 1/r
            prev = sample_path[j]
    bc = dict(sorted(bc.items(), key=lambda item: item[1], reverse=True))
    filtered_list = list(bc)
    return filtered_list

In [35]:
#saijruulsan girvan
def _without_most_central_edges(G, most_valuable_edge, edge_number):
    original_num_components = nx.number_connected_components(G)
    num_new_components = original_num_components
    limit = round(math.sqrt(G.number_of_edges()))
    edges = most_valuable_edge(G)
    while num_new_components <= original_num_components:
        edges_to_rmv = edges[0:limit]
        while edges_to_rmv:
            edge = edges_to_rmv[0]
            G.remove_edge(*edges_to_rmv[0])
            del edges_to_rmv[0]
        new_components = tuple(nx.connected_components(G))
        num_new_components = len(new_components)
        edges = most_valuable_edge(G)
    return new_components
def revised_girvan_quick(G):
    if G.number_of_edges() == 0:
        yield tuple(nx.connected_components(G))
        return
    # If no function is provided for computing the most valuable edge,
    # use the edge betweenness centrality.
#     if most_valuable_edge is None:

    def most_valuable_edge(G):
        apprx = bcApproxGen(G)
        return apprx

    # The copy of G here must include the edge weight data.
    g = G.copy().to_undirected()
    # Self-loops must be removed because their removal has no effect on
    # the connected components of the graph.
    g.remove_edges_from(nx.selfloop_edges(g))
    edge_number = g.number_of_edges()
    while g.number_of_edges() > 0:
        yield _without_most_central_edges(g, most_valuable_edge, g.number_of_edges())